In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
# url = 'https://github.com/kartikjindgar/NYU-DataScience-Bootcamp-Fall23/blob/main/Week6/train.csv'
data = pd.read_csv('train.csv')

In [5]:
print(data.columns)

Index(['id', 'timestamp', 'country', 'employment_status', 'job_title',
       'job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')


In [6]:
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [7]:
data.shape

(4277, 13)

In [8]:
data = data.drop(columns=['id','timestamp'])

In [9]:
null_columns = data.columns[data.isnull().any()]
null_columns

Index(['hours_per_week', 'telecommute_days_per_week',
       'is_education_computer_related'],
      dtype='object')

In [10]:
data['hours_per_week'] = data['hours_per_week'].fillna(data['hours_per_week'].median())
data['telecommute_days_per_week'] = data['telecommute_days_per_week'].fillna(data['telecommute_days_per_week'].median())

In [11]:
null_count = data['is_education_computer_related'].isnull().sum()
null_count

16

In [12]:
data = data.dropna(subset=['is_education_computer_related'])

In [13]:
null_columns = data.columns[data.isnull().any()]
null_columns

Index([], dtype='object')

In [14]:
data_train = data.copy()
data_train.head()

,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [15]:
data_train.shape

(4261, 11)

In [16]:
distinct_count = data['country'].nunique()
distinct_count

78

In [17]:
data = data.drop(columns=['country'])

In [18]:
data_train = data_train.drop(columns=['country'])

In [19]:
data_train.shape

(4261, 10)

In [20]:
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object' 
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [21]:
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [22]:
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,1,40.0,0.0,Bachelors (4 years),Yes,0,7187.743094
1,Full time,DBA,5.00000,0,40.0,5.0,Bachelors (4 years),Yes,0,10000.000000
2,Full time,DBA,1.00000,0,40.0,0.0,Masters,Yes,1,7000.000000
3,Full time,DBA,1.00000,0,40.0,2.0,Bachelors (4 years),No,0,8333.000000
4,Full time,Developer,3.00000,0,40.0,2.0,Masters,Yes,1,7137.000000


In [23]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True)
final_data.shape

(4261, 25)

In [24]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [25]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,...,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,5.000000,0,40.0,5.0,0,10000.000000,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
2,1.000000,0,40.0,0.0,1,7000.000000,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,1.000000,0,40.0,2.0,0,8333.000000,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,3.000000,0,40.0,2.0,1,7137.000000,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4273,4.140793,1,40.0,0.0,0,7033.845701,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4274,5.000000,0,40.0,0.0,0,7166.000000,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4275,4.000000,1,40.0,2.0,0,9583.000000,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [26]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (2982, 24)
Validation Set Dimensions: (1279, 24)


In [27]:
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols

['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [28]:
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [29]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,...,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
1011,0.278916,0,2.295988,-0.037409,0,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2416,0.785997,0,-0.494154,-0.637153,0,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
2684,0.278916,0,-0.494154,-0.637153,1,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
365,-1.242327,0,-0.494154,-0.037409,1,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
2033,2.307240,0,-2.168239,-0.637153,0,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,-0.249676,0,-0.325810,-0.637153,0,False,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False
4216,2.814321,0,-0.494154,-0.637153,1,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
417,-1.242327,0,-0.494154,-0.637153,1,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
2679,0.169351,1,-0.151298,-0.637153,0,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [30]:
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [31]:
reg.coef_

array([  205.81719566,    46.55332675,   186.02039623,   215.28737985,
          91.97940208,   -31.69374052,   233.0000356 , -2110.35104242,
        1135.74537931,   840.3216983 ,   532.71559582,   532.23935344,
         980.39673472,   810.94621379,    14.9704013 ,   909.67496112,
         695.04377767,  3608.42481501,   368.54410314,  -439.15489311,
         228.0390239 ,   189.06164251,  -320.61756556,  -177.49320956])

In [32]:
reg.intercept_

6144.982751303486

In [33]:
mean_absolute_error(y_train,reg.predict(X_train))

850.2878102813581

In [34]:
mean_squared_error(y_train,reg.predict(X_train))**0.5

1202.8901606546424

In [35]:
X_test[num_cols] = scaler.transform(X_test[num_cols])
y_pred = reg.predict(X_test)
print(mean_absolute_error(y_test,y_pred), mean_squared_error(y_test,y_pred)**0.5)

883.3306998387706 1244.60170637595


In [36]:
X_test.describe()

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications
count,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000
mean,0.050689,0.223612,-0.054071,-0.058510,0.519937
std,1.031508,0.416828,0.926552,0.963066,0.499798
min,-2.256489,0.000000,-2.168239,-0.637153,0.000000
25%,-0.528785,0.000000,-0.494154,-0.637153,0.000000
50%,0.142945,0.000000,-0.494154,-0.637153,1.000000
75%,0.278916,0.000000,0.091041,-0.037409,1.000000
max,2.814321,1.000000,2.295988,2.361569,1.000000


In [37]:
ridge = Ridge(alpha=1)
ridge.fit(X_train,y_train)
y_pred = ridge.predict(X_test)
print(mean_absolute_error(y_test,y_pred), mean_squared_error(y_test,y_pred)**0.5)

883.2399036570116 1244.5782961082998


In [38]:
lasso = Lasso(alpha=1)
lasso.fit(X_train,y_train)
y_pred = lasso.predict(X_test)
print(mean_absolute_error(y_test,y_pred), mean_squared_error(y_test,y_pred)**0.5)

882.7647019918758 1244.16355594489


Trees

In [39]:
decisiontree = DecisionTreeRegressor(max_depth = 10, min_samples_split = 5)
decisiontree.fit(X_train, y_train)

#evaluating train error
mean_absolute_error(y_train,decisiontree.predict(X_train))

626.1306140453892

In [47]:
import sys

max_depth_list = [2,3,4,5,6,7,8,9,10,11,12,20]
train_error = []
test_error =[]
min_test_error = sys.maxsize
depth_with_min_error = -1

for index, max_depth in enumerate(max_depth_list):

    decisiontree = DecisionTreeRegressor(max_depth = max_depth, min_samples_split = 2)
    decisiontree.fit(X_train, y_train)
    error = mean_absolute_error(y_test,decisiontree.predict(X_test))
    if(error < min_test_error):
        min_test_error = error
        depth_with_min_error = max_depth_list[index]
#     train_error.append(mean_absolute_error(y_train,decisiontree.predict(X_train)))
#     test_error.append(mean_absolute_error(y_test,decisiontree.predict(X_test)))
    
# print("train_error:")    
# print(train_error)
# print("test_error:")
# print(test_error)
print("depth with min error: ",depth_with_min_error)

depth with min error:  8
